<a href="https://colab.research.google.com/github/Heinsburg123/Chatbot/blob/main/DQN(PII).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from random import choice

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
Transition=namedtuple("Transition",("state","action","reward","nx_state"))
class Replay:
  def __init__(self,size):
    self.mem=deque([],maxlen=size)
  def push(self,*arg):
    self.mem.append(Transition(*arg))
  def get(self,batch_size):
    return random.sample(self.mem,batch_size)
  def get_index(self,index):
    return self.mem[index]
  def len(self):
    return len(self.mem)

In [32]:
class DQN(nn.Module):
  def __init__(self,n_observations,n_actions):
    super(DQN,self).__init__()
    self.layer1=nn.Linear(n_observations,400)
    self.layer2=nn.Linear(400,400)
    self.layer3=nn.Linear(400,n_actions)
  def forward(self,x):
    x=F.relu(self.layer1(x))
    x=F.relu(self.layer2(x))
    return self.layer3(x)

In [33]:
ANSWER=[(2,2),(9,2),(16,2),(2,9),(9,9),(16,9),(2,16),(9,16),()]

In [34]:
BATCH_SIZE=80
eps_st=1
eps_en=0.1
eps_decay=600
gamma=0.99
TAU=0.005
LR=0.001
n_actions=4
n_para=3
n_behind=5
expo=2
state=torch.tensor([])
n_obs=len(state)
policy=DQN(n_para*n_behind,n_actions).to(device)
target=DQN(n_para*n_behind,n_actions).to(device)
opti=optim.AdamW(policy.parameters(),lr=LR,amsgrad=True)
memory=Replay(10000)
step=0
turn=0
actions=([0,1],[1,0],[0,-1],[-1,0])


def select(state,cur_loc,turn,step):
  sample=random.random()
  eps_threshold=eps_en+(eps_st-eps_en)*math.exp(-1*step/eps_decay)
  lst=[]
  mask=[1,1,1,1]
  if(cur_loc[0]==19):
    mask[1]=0
  if(cur_loc[1]==19):
    mask[0]=0
  if(cur_loc[0]==0):
    mask[3]=0
  if(cur_loc[1]==0):
    mask[2]=0
  for i in range(4):
    if(mask[i]==1):
      lst.append(i)
  if(sample<=eps_threshold or turn<=10):
    return torch.tensor([[random.choice(lst)]], device=device,dtype=torch.long)
  else:
    val=policy(state)[0,lst].max(-1)[1]
    num=torch.tensor(lst[val.item()],device=device).view(1,1)
    return num


In [35]:
def optimize():
  if memory.len()<BATCH_SIZE:
    return
  transitions=memory.get(BATCH_SIZE)
  batch=Transition(*zip(*transitions))
  nx_state_batch=torch.cat(batch.nx_state)
  state_batch=torch.cat(batch.state)
  action_batch=torch.cat(batch.action)
  reward_batch=torch.cat(batch.reward)
  state_action_values=policy(state_batch).gather(1,action_batch)
  with torch.no_grad():
    nx_state_values=target(nx_state_batch).max(1)[0].unsqueeze(1)
  expect=nx_state_values*gamma+reward_batch.unsqueeze(1)
  crit=nn.SmoothL1Loss()
  loss=crit(state_action_values,expect)
  opti.zero_grad()
  loss.backward()
  torch.nn.utils.clip_grad_value_(policy.parameters(), 100)
  opti.step()


In [36]:
if torch.cuda.is_available():
  num_episodes = 1000
else:
  num_episodes = 1000
def training(map):
  step=0
  eps_st=0.9
  eps_en=0.1
  ans=np.max(map)
  dem=0
  for i in range(num_episodes):
    cur_loc=[19,0]
    state=torch.tensor([[]])
    state=torch.tensor(state,dtype=torch.float32,device=device).unsqueeze(0)
    check=np.zeros((20,20))
    turn=-1
    for zz in range(1600):
      turn=turn+1
      step=step+1
      action=select(state,cur_loc,turn,step)
      nx_loc=np.array(actions[action])+cur_loc
      passed=check[tuple(nx_loc)]
      reward=map[tuple(nx_loc)]-map[tuple(cur_loc)]-passed**2
      if(map[tuple(nx_loc)]==ans):
        reward=1e9-passed**2
      reward = torch.tensor([reward], device=device)
      pf=[map[tuple(nx_loc)]-map[tuple(cur_loc)],action,passed]
      nx_pos=torch.tensor([pf],dtype=torch.float32,device=device)
      if(turn>0):
        nx_state=torch.cat((state,nx_pos),-1)
      else:
        nx_state=torch.clone(nx_pos)
      if(nx_state.size()[1]==n_para*(n_behind+1)):
        nx_state=torch.clone(nx_state[:,n_para:])
      if(state.size()[1]==n_para*n_behind):
        memory.push(state,action,reward,nx_state)
      check[tuple(cur_loc)]=check[tuple(cur_loc)]+1
      cur_loc=nx_loc
      state=torch.clone(nx_state)
      optimize()
      target_net_state_dict = target.state_dict()
      policy_net_state_dict = policy.state_dict()
      for key in policy_net_state_dict:
          target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
      target.load_state_dict(target_net_state_dict)
      if(map[tuple(nx_loc)]==ans):
        print(dem)
        dem=dem+1
        break
  print(dem)


In [37]:
def testing(map):
  ans=np.max(map)
  state=torch.tensor([[]])
  state=torch.tensor(state,dtype=torch.float32,device=device).unsqueeze(0)
  check=np.zeros((20,20))
  cur_loc=np.array([19,0])
  ans=np.max(map)
  turn=-1
  for i in range(400):
    turn=turn+1
    action=select(state,cur_loc,turn,1e9)
    nx_loc=np.array(actions[action])+cur_loc
    if(map[tuple(nx_loc)]==ans):
      return turn
    passed=check[tuple(nx_loc)]
    pf=[map[tuple(nx_loc)]-map[tuple(cur_loc)],action,passed]
    nx_pos=torch.tensor([pf],dtype=torch.float32,device=device)
    if(turn>0):
      nx_state=torch.cat((state,nx_pos),-1)
    else:
      nx_state=torch.clone(nx_pos)
    if(nx_state.size()[1]==n_para*(n_behind+1)):
      nx_state=torch.clone(nx_state[:,n_para:])
    check[tuple(cur_loc)]=check[tuple(cur_loc)]+1
    cur_loc=nx_loc
    state=torch.clone(nx_state)


In [38]:
for i in range(1,2):
  board=(f'wall_1_{i}.csv')
  map=np.loadtxt(board,delimiter=',')
  for j in range(0,20):
    for z in range(0,20):
        map[j,z]=map[j,z]*10**7
        map[j,z]=map[j,z]**2
  training(map)

<ipython-input-36-b29db5f71e26>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state=torch.tensor(state,dtype=torch.float32,device=device).unsqueeze(0)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


KeyboardInterrupt: ignored

In [ ]:
for i in range(1,10):
  board=(f'wall_1_{i}.csv')
  map=np.loadtxt(board,delimiter=',')
  for j in range(0,20):
    for z in range(0,20):
        map[j,z]=map[j,z]*10**7
        map[j,z]=map[j,z]**2
  print(testing(map))
